In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states 
import gmaps

In [2]:
api_key = "1383ed6d1c178075bb6c33d544f4a17d700c9607"

In [3]:
c = Census(api_key, year=2016)

In [4]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})


# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,01,Alabama,4841164.0,38.6,44758.0,24736.0,868666.0,17.943329,3.810633
1,02,Alaska,736855.0,33.6,74444.0,34191.0,72826.0,9.883356,4.090221
2,04,Arizona,6728577.0,37.1,51340.0,26686.0,1165636.0,17.323663,3.715080
3,05,Arkansas,2968472.0,37.7,42336.0,23401.0,542431.0,18.273071,3.139326
4,06,California,38654206.0,36.0,63783.0,31458.0,6004257.0,15.533257,4.355868


In [31]:
gr_population = census_pd.groupby('State')
gr_population["Population"]


In [6]:
hospitalgen = pd.read_csv("csv_data/Hospital General Information.csv")
hospitalgen.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Hospital Type,Hospital Ownership,Emergency Services
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,Yes
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,Acute Care Hospitals,Government - Hospital District or Authority,Yes
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE,Acute Care Hospitals,Government - Hospital District or Authority,Yes
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,Acute Care Hospitals,Voluntary non-profit - Private,Yes
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,Acute Care Hospitals,Proprietary,Yes


In [7]:
hospitalstr = pd.read_csv("csv_data/Structural Measures - Hospital.csv")
hospitalstr.head()


,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Measure Name,Measure ID,Measure Response
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Able to receive lab results electronically,OP_12,Yes
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,"Able to track patients' lab results, tests, an...",OP_17,No
2,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,Able to receive lab results electronically,OP_12,Yes
3,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,"Able to track patients' lab results, tests, an...",OP_17,Yes
4,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE,Able to receive lab results electronically,OP_12,Yes


In [5]:
hospitaldetail = pd.read_csv("csv_data/HCAHPS - Hospital.csv")
hospitaldetail.head()



,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW


In [8]:
count = hospitaldetail["Hospital Name"].count()
count

4744

In [11]:
len(set(zip(hospitaldetail['State'],hospitaldetail['Hospital Name'])))

4744

In [13]:
state_counts = hospitaldetail["State"].value_counts()
state_counts.head()

TX    396
CA    345
FL    187
IL    174
NY    174
Name: State, dtype: int64

In [16]:
grouped_df = hospitaldetail.groupby('State')
grouped_df["Hospital Name"].count()

State
AK     22
AL     94
AR     77
AS      1
AZ     80
CA    345
CO     82
CT     32
DC      8
DE      8
FL    187
GA    130
GU      2
HI     22
IA    114
ID     42
IL    174
IN    118
KS    135
KY     88
LA    116
MA     64
MD     50
ME     32
MI    134
MN    125
MO    108
MP      1
MS     95
MT     62
NC    105
ND     45
NE     84
NH     24
NJ     63
NM     41
NV     37
NY    174
OH    160
OK    120
OR     60
PA    169
PR     53
RI     12
SC     61
SD     59
TN    106
TX    396
UT     44
VA     85
VI      2
VT     15
WA     81
WI    121
WV     50
WY     29
Name: Hospital Name, dtype: int64

In [34]:
census_data = c.acs5.get("Name": "GEO_ID",{'for': 'state:*'})

SyntaxError: invalid syntax (<ipython-input-34-60d16657046e>, line 1)